### 層の重みの共有

Functional APIのより重要な特徴の１つは、層のインスタンスを繰り返し再利用できること。<br>
層のインスタンスを2回目に呼び出すときには、新しいインスタンスを作成するのではなく、<br>
同じ重みを再利用することになる。<br>

複数の分岐を共有するモデルを構築できるようになる。<br>
それらの分岐は全て同じ知識を共有し、同じ演算を実行する。<br>
つまり、それらのモデルは同じ表現を共有し、様々な入力に基づいてそれらの表現を同時に学習する。<br>

### 例

２つの文章の意味的な類似性を評価するモデルがあるとする。<br>
このモデルは、入力として（比較の対象となる）２つの文章を受け取り、出力として0〜1のスコアを返す。<br>
この場合、0が文章に関連性がないとし、１は文章が全く同じであるか、同じ文章の組み換えであることを意味する。<br>

よって、２つの入力文章を処理するために２つのモデルを別々に学習することは合理的でない。<br>
それよりも単一のLSTM層で両方の入力を処理した方が良さそう。<br>
このLSTM層の表現（重み）は、両方の学習に基づいて同時に学習される。<br>

### 実装

In [ ]:
from keras import layers
from keras import Input
from keras.models import Model

# 単一のLSTM層を一度だけインスタンス化
lstm = layers.LSTM(32)

# モデルの左側の分岐を構築
# 入力はサイズが128のベクトルからなる可変長のシーケンス
left_input = Input(shape=(None, 128))
left_output = lstm(left_input)

# モデルの右側の分岐を構築
# 既存の層のインスタンスを呼び出すと、その重みを再利用できる
right_input = Input(shape=(None, 128))
right_output = lstm(right_input)

# 最後に分類器を構築
merged = layers.concatenate([left_output, right_output], axis=-1)
predictions = layers.Dense(1, activation='sigmoid')(merged)

# モデルのインスタンス化と訓練 : 
# このようなモデルを訓練するときには、
# LSTM層の重みが両方の入力に基づいて更新される
model = Model([left_input, right_input], predictions)
model.fit([left_data, right_data], targets)